In [ ]:
# Внимание!!! Важно, что бы файлы с данными и исполняемый файл находились в одной папке, 
# тогда пути к тестовым и тренировочным наборам будут содержать только имена файлов.
# 
# В пути к тренировочным и тестовым данным запрежается использовать абсалютную адресацию, 
# то есть адресацию, в которой присутствуют имена папок. Путь должен содержать только имя файла.
#
# Напоминание: под моделью машинного обучения понимаются все действия с исходными данными, 
# которые необходимо произвести, что бы сопоставить признаки целевому значению.

### Область работы 1 (библиотеки)

In [ ]:
# Данный блок в области 1 НЕ выполняется преподавателем
# 
# данный блок предназначен только для подключения необходимых библиотек
# запрещается подключать библиотеки в других блоках
# запрещается скрывать предупреждения системы
# установка дополнительных библиотек размещается прямо здесь (обязательно закоментированы)
# pip install

In [ ]:
import numpy as np
import pandas as pd
from IPython.display import display

from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression


from sklearn.model_selection import train_test_split, GridSearchCV

from sklearn.preprocessing import StandardScaler, RobustScaler, MinMaxScaler, Normalizer

from sklearn.pipeline import Pipeline, make_pipeline

from sklearn.metrics import classification_report

### Область работы 2 (поиск  модели .... )

In [ ]:
# Данный блок(и) НЕ выполняются преподавателем в области 2
# блок(и) предназначены для поиска лучшей модели 
# должен быть понятен и очевиден отбор параметров модели
# оставляйте свои комментарии и разъяснения
# 
# Запрещается размещать данные блоки за пределами обасти 2
# Все блоки данной области должны быть выполнены
#
# ЗАПРЕЩАЕТСЯ ИСПОЛЬЗОВАТЬ ТЕСТОВЫЙ НАБОР
#
# Путь к тренировочному набору
# 

path_train = 'train.csv' # содержит только имя файла, без имен папок !!!
wine_df = pd.read_csv(path_train)
wine_df.head()

In [ ]:
wine_df.describe()

In [ ]:
wine_df.info()

In [ ]:
# Проверка на дубликаты!

wine_duplicate = wine_df[wine_df.duplicated()]
wine_duplicate

In [ ]:
wine_df['good quality'] = [0 if i < 7 else 1 for i in wine_df['quality']]
wine_df.head()

In [ ]:
X = wine_df.drop(['quality', 'good quality'], axis = 1)
y = wine_df['good quality']
display(X, y)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42, stratify = y)

In [ ]:
scaling = [MinMaxScaler(), StandardScaler(), RobustScaler(), Normalizer()]

In [ ]:
# Параметры для ближайших соседей 

n_neighbors = np.arange(5, 20)
weights = ['uniform', 'distance']
metric = ['euclidean', 'manhattan', 'cosine']
p = np.arange(1, 3)

In [ ]:
# Ближайшие соседи

pipe_1 = Pipeline([('preprocessing', MinMaxScaler()),
                   ('classifier', KNeighborsClassifier())])

param_grid_1 = [{'preprocessing' : scaling,
               'classifier' : [KNeighborsClassifier()],
               'classifier__n_neighbors' : n_neighbors,
               'classifier__weights' : weights,
               'classifier__p' : p,
                'classifier__metric' : metric}]

grid_search_1 = GridSearchCV(pipe_1, param_grid_1, return_train_score = True)
grid_search_1.fit(X_train, y_train)
grid_search_1

In [ ]:
grid_search_1_result = pd.DataFrame(grid_search_1.cv_results_).sort_values(["rank_test_score",'std_test_score']).T
display(grid_search_1_result, grid_search_1.best_params_, grid_search_1.best_score_)

In [ ]:
# Переобученные ближайшие соседи

pipe_best_1 = make_pipeline(RobustScaler(), KNeighborsClassifier(n_neighbors = 10, p = 1, metric = 'cosine', weights = 'distance'))

pipe_best_1.fit(X_train, y_train)

y_pred_best_1 = pipe_best_1.predict(X_test)

print("Тренировака:", pipe_best_1.score(X_train, y_train), "Тест:", pipe_best_1.score(X_test, y_test))

print(classification_report(y_test, y_pred_best_1))

In [ ]:
grid_search_1_result[658]

In [ ]:
# Непереобученные ближайшие соседи 

pipe_best_1_1 = make_pipeline(RobustScaler(), KNeighborsClassifier(n_neighbors = 16, p = 1, metric = 'cosine', weights = 'uniform'))

pipe_best_1_1.fit(X_train, y_train)

y_pred_best_1_1 = pipe_best_1.predict(X_test)

print("Тренировака:", pipe_best_1_1.score(X_train, y_train), "Тест:", pipe_best_1_1.score(X_test, y_test))

print(classification_report(y_test, y_pred_best_1_1))

In [ ]:
# Наивный байес

pipe_2 = Pipeline([('preprocessing', StandardScaler()),
                    ('classifier', GaussianNB())])

param_grid_2 = [{'preprocessing' : scaling,
                'classifier' : [GaussianNB()]}]

grid_search_2 = GridSearchCV(pipe_2, param_grid_2, return_train_score = True)
grid_search_2.fit(X_train, y_train)
grid_search_2

In [ ]:
grid_search_2_result = pd.DataFrame(grid_search_2.cv_results_).sort_values(['rank_test_score','std_test_score']).T
display(grid_search_2_result, grid_search_2.best_params_, grid_search_2.best_score_)

In [ ]:
pipe_best_2 = make_pipeline(MinMaxScaler(), GaussianNB())

pipe_best_2.fit(X_train, y_train)

y_pred_best_2 = pipe_best_2.predict(X_test)

print("Тренировака:", pipe_best_2.score(X_train, y_train), "Тест:", pipe_best_2.score(X_test, y_test))

print(classification_report(y_test, y_pred_best_2))

In [ ]:
# Логистическая регрессия

pipe_3 = Pipeline([('preprocessing', StandardScaler()),
                    ('classifier', LogisticRegression())])

param_grid_3 = [{'preprocessing': scaling,
                    'classifier__penalty': ['l2'],
                    'classifier__solver': ['lbfgs', 'liblinear', 'newton-cg', 'newton-cholesky', 'sag', 'saga'],
                    'classifier__class_weight' : ['balanced']},
               {'preprocessing': scaling,
                    'classifier__penalty': ['l1'],
                    'classifier__solver': ['liblinear', 'saga'],
                    'classifier__class_weight' : ['balanced']},
               {'preprocessing': scaling,
                    'classifier__penalty': ['None'],
                    'classifier__solver': ['lbfgs','newton-cg', 'newton-cholesky', 'saga'],
                    'classifier__class_weight' : ['balanced']},
               {'preprocessing': scaling,
                    'classifier__penalty' : ['elasticnet'],
                    'classifier__solver' : ['saga'],
                    'classifier__class_weight' : ['balanced']}]

grid_search_3 = GridSearchCV(pipe_3, param_grid_3, return_train_score = True)
grid_search_3.fit(X_train, y_train)
grid_search_3

In [ ]:
grid_search_3_result = pd.DataFrame(grid_search_3.cv_results_).sort_values(["rank_test_score",'std_test_score']).T
display(grid_search_3_result, grid_search_3.best_params_, grid_search_3.best_score_)

In [ ]:
pipe_best_3 = make_pipeline(RobustScaler(), LogisticRegression(penalty = 'l2', solver = 'sag', class_weight = 'balanced'))

pipe_best_3.fit(X_train, y_train)

y_pred_best_3 = pipe_best_3.predict(X_test)

print("Тренировака:", pipe_best_3.score(X_train, y_train), "Тест:", pipe_best_3.score(X_test, y_test))

print(classification_report(y_test, y_pred_best_3))